<a href="https://colab.research.google.com/github/databyhuseyn/NLP/blob/main/build_makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [7]:
block_size = 3
X, Y = [], []
for w in words[:5]:
  # print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
... ---> a
..a ---> v
.av ---> a
ava ---> .
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [8]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [9]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [10]:
C = torch.randn((27, 2))
C

tensor([[-1.4368,  0.1860],
        [ 2.5550, -1.4237],
        [-2.0375,  0.4774],
        [ 0.1377, -0.8497],
        [ 0.8348,  1.8303],
        [ 1.2020,  0.4684],
        [ 0.4887, -0.5052],
        [-1.3784, -1.4674],
        [ 2.3329,  0.5171],
        [-0.4604,  0.6290],
        [ 1.1911, -0.4697],
        [-0.3607, -1.4488],
        [-0.3911,  0.9255],
        [ 1.3369, -0.4097],
        [-0.7166,  0.6591],
        [ 0.0963, -0.5682],
        [-0.3190, -0.0952],
        [ 0.7175, -0.6163],
        [-0.7017, -0.6974],
        [-0.4208, -0.1706],
        [-0.2117, -0.2877],
        [ 1.5767,  0.1579],
        [-0.7529, -1.4618],
        [ 0.0956, -1.3348],
        [ 0.1459,  0.1183],
        [ 1.7446,  0.3453],
        [-0.4839, -1.6415]])

In [11]:
F.one_hot(torch.tensor(5), num_classes = 27).float()

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [14]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [12]:
C[X].shape

torch.Size([32, 3, 2])

In [15]:
X[2]

tensor([ 0,  5, 13])

In [16]:
C[X[2]]

tensor([[-1.4368,  0.1860],
        [ 1.2020,  0.4684],
        [ 1.3369, -0.4097]])

In [17]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [18]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [20]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

torch.Size([32, 6])

In [21]:
emb[:, 0, :].shape, emb[:, 1, :].shape, emb[:, 2, :].shape

(torch.Size([32, 2]), torch.Size([32, 2]), torch.Size([32, 2]))

In [22]:
# automate previous process
torch.cat(torch.unbind(emb, 1), 1)

tensor([[-1.4368,  0.1860, -1.4368,  0.1860, -1.4368,  0.1860],
        [-1.4368,  0.1860, -1.4368,  0.1860,  1.2020,  0.4684],
        [-1.4368,  0.1860,  1.2020,  0.4684,  1.3369, -0.4097],
        [ 1.2020,  0.4684,  1.3369, -0.4097,  1.3369, -0.4097],
        [ 1.3369, -0.4097,  1.3369, -0.4097,  2.5550, -1.4237],
        [-1.4368,  0.1860, -1.4368,  0.1860, -1.4368,  0.1860],
        [-1.4368,  0.1860, -1.4368,  0.1860,  0.0963, -0.5682],
        [-1.4368,  0.1860,  0.0963, -0.5682, -0.3911,  0.9255],
        [ 0.0963, -0.5682, -0.3911,  0.9255, -0.4604,  0.6290],
        [-0.3911,  0.9255, -0.4604,  0.6290, -0.7529, -1.4618],
        [-0.4604,  0.6290, -0.7529, -1.4618, -0.4604,  0.6290],
        [-0.7529, -1.4618, -0.4604,  0.6290,  2.5550, -1.4237],
        [-1.4368,  0.1860, -1.4368,  0.1860, -1.4368,  0.1860],
        [-1.4368,  0.1860, -1.4368,  0.1860,  2.5550, -1.4237],
        [-1.4368,  0.1860,  2.5550, -1.4237, -0.7529, -1.4618],
        [ 2.5550, -1.4237, -0.7529, -1.4

In [24]:
a = torch.arange(18)

In [25]:
a.shape

torch.Size([18])

In [26]:
a.view(3, 3, 2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [27]:
a.storage()

<ipython-input-27-d39b3459ecdd>:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [28]:
emb.view(32, 6)       # view is more efficient

tensor([[-1.4368,  0.1860, -1.4368,  0.1860, -1.4368,  0.1860],
        [-1.4368,  0.1860, -1.4368,  0.1860,  1.2020,  0.4684],
        [-1.4368,  0.1860,  1.2020,  0.4684,  1.3369, -0.4097],
        [ 1.2020,  0.4684,  1.3369, -0.4097,  1.3369, -0.4097],
        [ 1.3369, -0.4097,  1.3369, -0.4097,  2.5550, -1.4237],
        [-1.4368,  0.1860, -1.4368,  0.1860, -1.4368,  0.1860],
        [-1.4368,  0.1860, -1.4368,  0.1860,  0.0963, -0.5682],
        [-1.4368,  0.1860,  0.0963, -0.5682, -0.3911,  0.9255],
        [ 0.0963, -0.5682, -0.3911,  0.9255, -0.4604,  0.6290],
        [-0.3911,  0.9255, -0.4604,  0.6290, -0.7529, -1.4618],
        [-0.4604,  0.6290, -0.7529, -1.4618, -0.4604,  0.6290],
        [-0.7529, -1.4618, -0.4604,  0.6290,  2.5550, -1.4237],
        [-1.4368,  0.1860, -1.4368,  0.1860, -1.4368,  0.1860],
        [-1.4368,  0.1860, -1.4368,  0.1860,  2.5550, -1.4237],
        [-1.4368,  0.1860,  2.5550, -1.4237, -0.7529, -1.4618],
        [ 2.5550, -1.4237, -0.7529, -1.4

In [29]:
emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T

In [30]:
h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

In [31]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [32]:
logits = h @ W2 + b2

In [33]:
logits.shape

torch.Size([32, 27])

In [34]:
counts = logits.exp()   # softmax

In [35]:
prob = counts / counts.sum(1, keepdims = True)    # softmax

In [36]:
prob.shape

torch.Size([32, 27])

In [37]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [38]:
prob[torch.arange(32), Y]

tensor([3.7513e-12, 5.2550e-11, 7.6791e-04, 1.1204e-01, 3.8322e-11, 5.3960e-08,
        1.0347e-11, 1.0711e-01, 1.2560e-06, 9.4647e-01, 1.0930e-03, 9.8121e-12,
        2.3687e-13, 4.6957e-16, 9.0441e-06, 2.8835e-09, 2.1015e-01, 6.8291e-09,
        1.6162e-16, 1.9351e-02, 9.9999e-01, 2.9265e-12, 2.8464e-07, 6.6131e-11,
        1.8942e-10, 1.1982e-11, 6.2864e-06, 4.8331e-10, 2.4377e-04, 1.0085e-02,
        2.0010e-08, 1.0222e-09])